# RNN Assignment

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from string import punctuation
from keras.models import load_model,Model
from keras.layers import *
import keras
import numpy as np
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import ReduceLROnPlateau
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
train = pd.read_csv("./input/train.csv")#[:500]
test = pd.read_csv("./input/test.csv")[:1]

In [3]:
test.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...


In [4]:
trainLabel=np.array(train['is_duplicate'])#[:500]
train.head()





,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
# Check for any null values
print(train.isnull().sum())
print(test.isnull().sum())

id              0
qid1            0
qid2            0
question1       0
question2       2
is_duplicate    0
dtype: int64
test_id      0
question1    0
question2    0
dtype: int64


In [6]:
# Add the string 'empty' to empty strings
train = train.fillna('empty')
test = test.fillna('empty')

In [7]:
# Preview some of the pairs of questions
a = 0 
for i in range(a,a+10):
    print(train.question1[i])
    print(train.question2[i])
    print()

What is the step by step guide to invest in share market in india?
What is the step by step guide to invest in share market?

What is the story of Kohinoor (Koh-i-Noor) Diamond?
What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?

How can I increase the speed of my internet connection while using a VPN?
How can Internet speed be increased by hacking through DNS?

Why am I mentally very lonely? How can I solve it?
Find the remainder when [math]23^{24}[/math] is divided by 24,23?

Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?
Which fish would survive in salt water?

Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?
I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?

Should I buy tiago?
What keeps childern active and far from phone and video games?

How can I be a good geologist?
What should I do to be a great geologist?

When do you use シ instea

In [8]:
stop_words = ['the','a','an','and','but','if','or','because','as','what','which','this','that','these','those','then',
              'just','so','than','such','both','through','about','for','is','of','while','during','to','What','Which',
              'Is','If','While','This']

In [9]:
def text_to_wordlist(text, remove_stop_words=False, stem_words=False):
    # Clean the text, with the option to remove stop_words and to stem words.

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text = re.sub(r"what's", " what is ", text)
    text = re.sub(r"What's", " what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "I am", text)
    text = re.sub(r" m ", " am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"60k", " 60000 ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e-mail", "email", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"quikly", "quickly", text)
    text = re.sub(r" usa ", " America ", text)
    text = re.sub(r" USA ", " America ", text)
    text = re.sub(r" u s ", " America ", text)
    text = re.sub(r" uk ", " England ", text)
    text = re.sub(r" UK ", " England ", text)
    text = re.sub(r"india", "India", text)
    text = re.sub(r"switzerland", "Switzerland", text)
    text = re.sub(r"china", "China", text)
    text = re.sub(r"chinese", "Chinese", text) 
    text = re.sub(r"imrovement", "improvement", text)
    text = re.sub(r"intially", "initially", text)
    text = re.sub(r"quora", "Quora", text)
    text = re.sub(r" dms ", "direct messages ", text)  
    text = re.sub(r"demonitization", "demonetization", text) 
    text = re.sub(r"actived", "active", text)
    text = re.sub(r"kms", " kilometers ", text)
    text = re.sub(r"KMs", " kilometers ", text)
    text = re.sub(r" cs ", " computer science ", text) 
    text = re.sub(r" upvotes ", " up votes ", text)
    text = re.sub(r" iPhone ", " phone ", text)
    text = re.sub(r"\0rs ", " rs ", text) 
    text = re.sub(r"calender", "calendar", text)
    text = re.sub(r"ios", "operating system", text)
    text = re.sub(r"gps", "GPS", text)
    text = re.sub(r"gst", "GST", text)
    text = re.sub(r"programing", "programming", text)
    text = re.sub(r"bestfriend", "best friend", text)
    text = re.sub(r"dna", "DNA", text)
    text = re.sub(r"III", "3", text) 
    text = re.sub(r"the US", "America", text)
    text = re.sub(r"Astrology", "astrology", text)
    text = re.sub(r"Method", "method", text)
    text = re.sub(r"Find", "find", text) 
    text = re.sub(r"banglore", "Banglore", text)
    text = re.sub(r" J K ", " JK ", text)
    
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation])
    
    # Optionally, remove stop words
    if remove_stop_words:
        text = text.split()
        text = [w for w in text if not w in stop_words]
        text = " ".join(text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [10]:
def process_questions(question_list, questions, question_list_name, dataframe):
    '''transform questions and display progress'''
    for question in questions:
        question_list.append(text_to_wordlist(question).lower())
        if len(question_list) % 100000 == 0:
            progress = len(question_list)/len(dataframe) * 100
            print("{} is {}% complete.".format(question_list_name, round(progress, 1)))

In [11]:
train_question1 = []
process_questions(train_question1, train.question1, 'train_question1', train)

train_question1 is 24.7% complete.
train_question1 is 49.5% complete.
train_question1 is 74.2% complete.
train_question1 is 98.9% complete.


In [12]:
train_question2 = []
process_questions(train_question2, train.question2, 'train_question2', train)

train_question2 is 24.7% complete.
train_question2 is 49.5% complete.
train_question2 is 74.2% complete.
train_question2 is 98.9% complete.


In [13]:
test_question1 = []
process_questions(test_question1, test.question1, 'test_question1', test)

In [14]:
test_question2 = []
process_questions(test_question2, test.question2, 'test_question2', test)

In [15]:
# Preview some transformed pairs of questions
a = 0 
for i in range(a,a+10):
    print(train_question1[i])
    print(train_question2[i])
    print()

what is the step by step guide to invest in share market in india 
what is the step by step guide to invest in share market 

what is the story of kohinoor koh i noor diamond 
what would happen if the indian government stole the kohinoor koh i noor diamond back 

how can i increase the speed of my internet connection while using a vpn 
how can internet speed be increased by hacking through dns 

why am i mentally very lonely how can i solve it 
find the remainder when math 23 24 math is divided by 24 23 

which one dissolve in water quickly sugar salt methane and carbon di oxide 
which fish would survive in salt water 

astrology i am a capricorn sun cap moon and cap rising what does that say about me 
i am a triple capricorn sun moon and ascendant in capricorn what does this say about me 

should i buy tiago 
what keeps childern active and far from phone and video games 

how can i be a good geologist 
what should i do to be a great geologist 

when do you use instead of 
when do you 

In [16]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_SEQUENCE_LENGTH = 30
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 100 

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(train_question1+train_question2+test_question1+test_question2)
sequencestr1 = tokenizer.texts_to_sequences(train_question1)
sequencestr2 = tokenizer.texts_to_sequences(train_question2)
sequencestt1 = tokenizer.texts_to_sequences(test_question1)
sequencestt2 = tokenizer.texts_to_sequences(test_question2)
#print(tokenizer)
#print(sequences)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

datatr1 = pad_sequences(sequencestr1, maxlen=MAX_SEQUENCE_LENGTH)
datatr2 = pad_sequences(sequencestr2, maxlen=MAX_SEQUENCE_LENGTH)
datatt1 = pad_sequences(sequencestt1, maxlen=MAX_SEQUENCE_LENGTH)
datatt2 = pad_sequences(sequencestt2, maxlen=MAX_SEQUENCE_LENGTH)

print('Shape of data tensor:', datatr1.shape)
print(datatr1[0])


C:\Users\HuaSheng\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\keras\preprocessing\text.py:90: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 85424 unique tokens.
Shape of data tensor: (404290, 30)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    2    3    1 1221   60 1221 2569    7  578    8  761  381    8   36]


In [17]:
import os

embeddings_index = {}
f = open(os.path.join('./', 'glove6B100d.txt'), encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [18]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
print(embedding_matrix.shape)

(85425, 100)


In [19]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [20]:


trainLabel = to_categorical(trainLabel,2)
trainLabel[:10]

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.]])

In [21]:
'''
def processBlk(sequenceInp):
    embedded_sequences = embedding_layer(sequenceInp)
    x = Conv1D(128, 5, activation='relu',padding='same')(embedded_sequences)
    x = MaxPooling1D(2)(x)
    x = Conv1D(128, 5, activation='relu',padding='same')(x)
    x = MaxPooling1D(2)(x)
    x = Conv1D(128, 5, activation='relu',padding='same')(x)
    x = MaxPooling1D(2)(x)  # global max pooling
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    return(x)'''
''' x = Bidirectional(LSTM(32,return_sequences=True))(embedded_sequences)
    x = Bidirectional(LSTM(32,return_sequences=True))(x)'''

biLSTM1 = Bidirectional(LSTM(50,return_sequences=True))
biLSTM2 = Bidirectional(LSTM(50))
dense1 =  Dense(128, activation='relu')
dense2 =  Dense(128, activation='relu')
def processBlk(sequenceInp):
    embedded_sequences = embedding_layer(sequenceInp)
    x = biLSTM1(embedded_sequences)
    x = Dropout(0.2)(x)
    x = biLSTM2(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = dense1(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = dense2(x)
    return(x)

sequence_inputtr1 = Input(shape=(MAX_SEQUENCE_LENGTH,),name = 'tr1Inp', dtype='int32')
x1 = processBlk(sequence_inputtr1)
sequence_inputtr2 = Input(shape=(MAX_SEQUENCE_LENGTH,),name = 'tr2Inp', dtype='int32')
x2 = processBlk(sequence_inputtr2)

subtract_layer = Lambda(lambda inputs: inputs[0] - inputs[1])

x = subtract_layer([x1, x2])
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
preds = Dense(2, activation='softmax')(x)


model = Model([sequence_inputtr1 ,sequence_inputtr2 ], preds)
model.summary()

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
tr1Inp (InputLayer)              (None, 30)            0                                            
____________________________________________________________________________________________________
tr2Inp (InputLayer)              (None, 30)            0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 30, 100)       8542500     tr1Inp[0][0]                     
                                                                   tr2Inp[0][0]                     
____________________________________________________________________________________________________
bidirectional_1 (Bidirectional)  (None, 30, 100)       60400       embedding_1[0][0]       

In [22]:
# happy learning!
model.fit([datatr1,datatr2], trainLabel, validation_split=0.1,shuffle=True,
          epochs=5, batch_size=512)

Train on 363861 samples, validate on 40429 samples
Epoch 1/5
363861/363861 [==============================] - 694s - loss: 0.5656 - acc: 0.6977 - val_loss: 0.5990 - val_acc: 0.6751
Epoch 2/5
363861/363861 [==============================] - 707s - loss: 0.5013 - acc: 0.7488 - val_loss: 0.4856 - val_acc: 0.7541
Epoch 3/5
363861/363861 [==============================] - 758s - loss: 0.4714 - acc: 0.7674 - val_loss: 0.4592 - val_acc: 0.7724
Epoch 4/5
363861/363861 [==============================] - 730s - loss: 0.4532 - acc: 0.7801 - val_loss: 0.4449 - val_acc: 0.7838
Epoch 5/5
363861/363861 [==============================] - 710s - loss: 0.4401 - acc: 0.7874 - val_loss: 0.4531 - val_acc: 0.7779


In [24]:
model.save('./quora100dglove.h5')

In [23]:
keras.__version__

'2.0.5'